In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data

In [3]:
daily_df = pd.read_csv("dataset/stations_daily.csv")

In [4]:
hourly_df = pd.read_csv("dataset/stations_hourly.csv")

In [5]:
rsinaica_df = pd.read_csv("dataset/stations_rsinaica.csv")

# จัดการกับข้อมูลรายงานฝุ่นรายวันก่อน

we will use this dataframe to calculate the yearly statistic of ฝุ่น

In [6]:
daily_df.head()

,datetime,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
0,2015-04-02,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.361000,NaN,NaN
1,2015-05-21,32,0.454967,0.347400,0.059194,0.029176,NaN,44.879091,0.029525,0.053121,...,NaN,NaN,NaN,570.775455,0.296364,NaN,7.871636,16.792818,NaN,NaN
2,2015-05-22,32,0.077821,0.389323,0.019158,0.051903,NaN,42.506000,0.009589,0.014215,...,NaN,NaN,NaN,558.360000,0.002750,NaN,384.788079,17.018949,NaN,NaN
3,2015-05-23,32,0.018036,0.387729,0.019711,0.021092,0.144603,54.734437,0.009863,0.013436,...,NaN,NaN,NaN,573.282187,0.038750,NaN,252.605975,18.575000,NaN,NaN
4,2015-05-24,32,0.024935,0.398042,0.024757,0.009568,0.494143,58.374574,0.012397,0.010337,...,NaN,NaN,NaN,572.169574,0.003404,NaN,259.231664,16.781064,NaN,NaN


In [7]:
daily_df.shape

(231592, 27)

Let's add year and month columns to daily_df 

In [8]:
daily_df.insert(1, "year", daily_df["datetime"].apply(lambda x: x.split("-")[0]))
daily_df.insert(1, "month", daily_df["datetime"].apply(lambda x: x.split("-")[1]))

convert month and year to int \
then sort values by month and year

In [9]:
daily_df["month"] = pd.to_numeric(daily_df["month"])
daily_df["year"] = pd.to_numeric(daily_df["year"])

daily_df.sort_values(by=["year", "month"], inplace = True)
daily_df.head()

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
59820,2000-01-01,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59821,2000-01-02,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59822,2000-01-03,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59823,2000-01-04,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59824,2000-01-05,1,2000,102,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


count the values exist each year

In [10]:
groupDaily_by_year = daily_df.groupby("year")
groupDaily_by_year.count()
# we will keep a year if PM2.5 have data > 30 %

def keep_year(group):
    return ((group.count()["PM2.5"] * 100) / len(groupDaily_by_year) > 30)
groupDaily_by_year.count()

,datetime,month,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,...,HCT,HRI,IUV,PB,PP,PST,RS,TMPI,UVA,XIL
year,,,,,,,,,,,,,,,,,,,,,
2000,1678,1678,1678,0,0,0,0,0,0,1678,...,0,0,0,0,0,0,0,0,0,0
2001,1658,1658,1658,0,0,0,0,0,0,1658,...,0,0,0,0,0,0,0,0,0,0
2002,826,826,826,0,0,0,0,0,0,826,...,0,0,0,0,0,0,0,0,0,0
2006,321,321,321,0,0,0,0,0,0,321,...,0,0,0,0,0,0,0,0,0,0
2007,44,44,44,0,0,0,0,0,0,44,...,0,0,0,0,0,0,0,0,0,0
2008,43,43,43,0,0,0,0,0,0,43,...,0,0,0,0,0,0,0,0,0,0
2009,43,43,43,0,0,0,0,0,0,43,...,0,0,0,0,0,0,0,0,0,0
2010,10417,10417,10417,3814,7612,151,8235,6816,151,1069,...,0,0,0,140,0,0,0,0,0,0
2011,11315,11315,11315,5374,8473,61,10057,6298,65,1836,...,0,29,0,36,27,0,0,25,0,0


filter data

In [11]:
daily_df = groupDaily_by_year.filter(keep_year)
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226979 entries, 12133 to 231591
Data columns (total 29 columns):
datetime      226979 non-null object
month         226979 non-null int64
year          226979 non-null int64
station_id    226979 non-null int64
PM2.5         161698 non-null float64
PM10          180481 non-null float64
NOx           112881 non-null float64
O3            192957 non-null float64
CO            175493 non-null float64
HR            123285 non-null float64
NO            113541 non-null float64
NO2           169286 non-null float64
TMP           127015 non-null float64
BEN           775 non-null float64
CH4           270 non-null float64
CN            595 non-null float64
CO2           56 non-null float64
H2S           280 non-null float64
HCNM          279 non-null float64
HCT           0 non-null float64
HRI           10205 non-null float64
IUV           4936 non-null float64
PB            91738 non-null float64
PP            75346 non-null float64
PST      

next we will drop column which have few data

In [12]:
daily_df.isna().sum()* 100 / len(daily_df)

#show % of Nan values in each column, we will drop at 51 %

datetime        0.000000
month           0.000000
year            0.000000
station_id      0.000000
PM2.5          28.760810
PM10           20.485596
NOx            50.268086
O3             14.989052
CO             22.683156
HR             45.684403
NO             49.977311
NO2            25.417770
TMP            44.041079
BEN            99.658559
CH4            99.881046
CN             99.737861
CO2            99.975328
H2S            99.876641
HCNM           99.877081
HCT           100.000000
HRI            95.503989
IUV            97.825349
PB             59.583045
PP             66.804859
PST           100.000000
RS             66.733927
TMPI           84.851021
UVA            92.911238
XIL            99.658559
dtype: float64

Too many Nan values OMG

In [13]:
for col in daily_df:
    if daily_df[col].isna().sum()* 100 / len(daily_df) > 51:
        daily_df.drop(col, axis=1,inplace=True)

daily_df = daily_df.reset_index(drop = True)
daily_df

/usr/lib/python3/dist-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
0,2010-01-20,1,2010,68,NaN,50.444444,0.019147,NaN,NaN,35.205704,0.007604,0.011870,22.005545
1,2010-01-21,1,2010,68,NaN,67.500000,0.042956,NaN,NaN,42.276420,0.028849,0.014479,16.882448
2,2010-01-22,1,2010,68,NaN,67.291667,NaN,NaN,NaN,NaN,NaN,0.015071,NaN
3,2010-01-23,1,2010,68,NaN,69.500000,NaN,NaN,NaN,NaN,NaN,0.014679,NaN
4,2010-01-24,1,2010,68,NaN,47.500000,NaN,NaN,NaN,NaN,NaN,0.013279,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226974,2021-12-27,12,2021,426,28.956522,67.375000,0.036429,0.030750,1.684583,56.541667,NaN,0.024762,21.660833
226975,2021-12-28,12,2021,426,20.818182,55.833333,0.037508,0.026792,1.700000,44.375000,NaN,0.023225,23.731250
226976,2021-12-29,12,2021,426,16.857143,51.458333,0.045588,0.017083,1.732917,37.875000,NaN,0.026500,22.751250
226977,2021-12-30,12,2021,426,21.555556,51.750000,0.037229,0.029833,1.520000,42.916667,NaN,0.023929,22.988333


In [14]:
mean_y = daily_df.set_index("year").groupby("year")["PM2.5"].mean()[3:]
mean_y

year
2013    2.870717e+01
2014    2.646882e+01
2015    3.329578e+01
2016    5.457275e+01
2017    5.492572e+01
2018    4.002737e+01
2019    2.884864e+01
2020    1.788649e+34
2021    4.701081e+02
Name: PM2.5, dtype: float64

In [27]:
for y in daily_df["year"].unique()[3:]:
    daily_df.update(daily_df.loc[daily_df["year"]==y,"PM2.5"].fillna(mean_y[y]))
daily_df[daily_df["year"]==2014]

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
49005,2014-01-01,1,2014,53,64.000000,97.000000,NaN,0.009333,1.400000,NaN,0.025067,0.025067,NaN
49006,2014-01-08,1,2014,53,36.222222,102.555556,NaN,0.015727,0.845455,NaN,0.025000,0.025000,NaN
49007,2014-01-09,1,2014,53,30.043478,64.652174,NaN,0.014739,0.570000,NaN,0.018143,0.018143,NaN
49008,2014-01-10,1,2014,53,24.000000,65.208333,NaN,0.021208,0.766667,NaN,0.021337,0.021337,NaN
49009,2014-01-11,1,2014,53,25.833333,72.791667,NaN,0.017957,0.320000,NaN,0.021367,0.021367,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66267,2014-12-27,12,2014,388,26.468820,NaN,NaN,0.009625,1.258750,NaN,NaN,NaN,NaN
66268,2014-12-28,12,2014,388,26.468820,NaN,NaN,0.013667,1.224583,NaN,NaN,NaN,NaN
66269,2014-12-29,12,2014,388,26.468820,NaN,NaN,0.015458,1.425833,NaN,NaN,NaN,NaN
66270,2014-12-30,12,2014,388,26.468820,NaN,NaN,0.014917,1.664167,NaN,NaN,0.015333,NaN


In [24]:
daily_df

,datetime,month,year,station_id,PM2.5,PM10,NOx,O3,CO,HR,NO,NO2,TMP
0,2010-01-20,1,2010,68,NaN,50.444444,0.019147,NaN,NaN,35.205704,0.007604,0.011870,22.005545
1,2010-01-21,1,2010,68,NaN,67.500000,0.042956,NaN,NaN,42.276420,0.028849,0.014479,16.882448
2,2010-01-22,1,2010,68,NaN,67.291667,NaN,NaN,NaN,NaN,NaN,0.015071,NaN
3,2010-01-23,1,2010,68,NaN,69.500000,NaN,NaN,NaN,NaN,NaN,0.014679,NaN
4,2010-01-24,1,2010,68,NaN,47.500000,NaN,NaN,NaN,NaN,NaN,0.013279,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226974,2021-12-27,12,2021,426,28.956522,67.375000,0.036429,0.030750,1.684583,56.541667,NaN,0.024762,21.660833
226975,2021-12-28,12,2021,426,20.818182,55.833333,0.037508,0.026792,1.700000,44.375000,NaN,0.023225,23.731250
226976,2021-12-29,12,2021,426,16.857143,51.458333,0.045588,0.017083,1.732917,37.875000,NaN,0.026500,22.751250
226977,2021-12-30,12,2021,426,21.555556,51.750000,0.037229,0.029833,1.520000,42.916667,NaN,0.023929,22.988333


# hourly_df

In [17]:
hourly_df.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:3107e3c5c417bc1eb1dcad17606b18aa3d3...
1,size 559250617


In [18]:
# for y in daily_df["year"].unique()[3:]:
#     value = (mean_y[y])
#     print(y)
#     (daily_df[daily_df["year"]==y]["PM2.5"].fillna(value, inplace = True))


In [19]:
daily_df[daily_df["year"] == 2010]["PM10"]

0        50.444444
1        67.500000
2        67.291667
3        69.500000
4        47.500000
           ...    
10412          NaN
10413          NaN
10414          NaN
10415          NaN
10416          NaN
Name: PM10, Length: 10417, dtype: float64

# rsinaica_df

In [20]:
hourly_df.drop_duplicates(["datetime", "station_id"])
hourly_df.sort_values("datetime")

KeyError: Index(['datetime', 'station_id'], dtype='object')

In [ ]:
rsinaica_df.info()

make new dataframe caontain station_id and network_name(city)

In [ ]:
data = {"station_id":rsinaica_df["station_id"],
        "network_name":rsinaica_df["network_name"]}
new_df = pd.DataFrame(data)

In [ ]:
new_df.fillna("Unknown")

#  prepare data for คำนวณ อับดับเมือง
merge new_df and daily_df

In [ ]:
daily_station = pd.merge(daily_df, new_df,on="station_id")

In [ ]:
daily_station.sort_values(by=["year", "month"], inplace = True)

In [ ]:
daily_station